In [1]:
!pip install emoji==0.6.0
!pip install transformers
!pip install soynlp
!pip install pytorch_lightning
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=b669a51487502b8bc005b5959ce3922dd6c160b8fa6efd3247a80534ffc8451b
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 28.

In [2]:
import transformers
import emoji
import soynlp
import pytorch_lightning

In [3]:
import os
import pandas as pd
import numpy as np

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

In [107]:
from utils.emotion_model import Model
from utils.infer_emotion import infer
from utils.load_data import load_dataset, idx2emoticon, emoticon2idx, convert_sentence_emotion, add_emotion

In [5]:
from sentence_transformers import SentenceTransformer, util

from sklearn.datasets import make_classification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls '/content/drive/MyDrive'

 3-1	 4-1	    'Colab Notebooks'		 etc    인지프.gslides
 3-1.5	 Bbox_0001   DNN_HW3_1017		 URP
 3-2	 Classroom   epoch9-val_acc0.7202.ckpt	 교재


In [16]:
emotion_model_path = '/content/drive/MyDrive/epoch9-val_acc0.7202.ckpt'
sentence_model_path = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
emoticon_sentence_path = '/content/dataset/sentence_emoticon.txt'
idx2emoticon_path = '/content/dataset/idx2emoticon.txt'
emoticon2idx_path = '/content/dataset/emoticon2idx.txt'

In [12]:
idx2emoticon = idx2emoticon(idx2emoticon_path)
emoticon2idx = emoticon2idx(emoticon2idx_path)

In [13]:
emotion_model = Model.load_from_checkpoint(emotion_model_path)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.weight', 'classifier.

In [14]:
sentence_model = SentenceTransformer(sentence_model_path)

In [17]:
train_sentences, train_emotions, train_labels = load_dataset(emoticon_sentence_path)
train_sentences = convert_sentence_emotion(sentence_model, train_emotions, train_sentences)

In [18]:
knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute', metric='cosine')
knn.fit(train_sentences, train_labels)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_neighbors=3)

In [19]:
input_sentence = ['오늘 아침이 좀 개운한데?']

In [81]:
sentence_emotion = infer(emotion_model, input_sentence)
sentence_emotion = np.argmax(sentence_emotion.cpu().detach().numpy())
sentence_emotion

2

In [97]:
# from utils.load_data import *
sentence_vector = add_emotion(sentence_model, sentence_emotion, input_sentence)

fwefwefew
0.7025553


In [109]:
emoticon_idx = knn.predict([sentence_vector])

In [110]:
selected = [idx for idx, i in enumerate(knn.predict_proba([sentence_vector]).reshape(-1)) if i!=0]
selected

[0, 46, 55]

In [38]:
#Check all neighbors
for i in selected:
    print('Recommended emoticon: ' + idx2emoticon[int(i)])

Recommended emoticon: ⸜(*ˊᗜˋ*)⸝
Recommended emoticon: ┏༼ ◉ ╭╮ ◉༽┓
Recommended emoticon: o(╥﹏╥)o


In [23]:
print('Recommended emoticon: ' + idx2emoticon[int(emoticon_idx)])

Recommended emoticon: ⸜(*ˊᗜˋ*)⸝
